In [ ]:
import h2o

In [ ]:
import h2o_cloud_extensions as hce

In [ ]:
# Go to https://internal.dedicated.h2o.ai/auth/get-platform-token
hce.settings.connection.refresh_token = "TOKEN_THAT_YOU_RECIEVED_AFTER_AUTHENTICATION"

In [ ]:
hce.settings.connection.client_id = 'hac-platform-public'
hce.settings.connection.token_endpoint_url = 'https://auth.internal.dedicated.h2o.ai/auth/realms/hac/protocol/openid-connect/token'
hce.settings.mlops.api_url = 'https://mlops-api.internal.dedicated.h2o.ai'
hce.settings.mlops.project_name = 'h2o-3-test'
hce.settings.mlops.project_description = 'Testing project for h2o3 cloud extensions'
hce.settings.mlops.deployment_environment = 'DEV'
hce.settings.mlops.grid_search.automatic_publishing = False
hce.settings.mlops.grid_search.automatic_deployment = False


In [ ]:
h2o.init()

In [ ]:
# Train model
from h2o.estimators import H2OGradientBoostingEstimator
from h2o.grid.grid_search import H2OGridSearch
prostate = h2o.import_file("http://s3.amazonaws.com/h2o-public-test-data/smalldata/prostate/prostate.csv")
prostate["CAPSULE"] = prostate["CAPSULE"].asfactor()
predictors = ["ID","AGE","RACE","DPROS","DCAPS","PSA","VOL","GLEASON"]
response = "CAPSULE"
gbm_params = {'learn_rate': [i * 0.01 for i in range(1, 11)],
                'max_depth': list(range(2, 11)),
                'sample_rate': [i * 0.1 for i in range(5, 11)],
                'col_sample_rate': [i * 0.1 for i in range(1, 11)]}

search_criteria = {'strategy': 'RandomDiscrete', 'max_models': 5, 'seed': 2}

# Train and validate a random grid of GBMs
gbm_grid = H2OGridSearch(model=H2OGradientBoostingEstimator,
                          hyper_params=gbm_params,
                          search_criteria=search_criteria)
gbm_grid.train(x=predictors, y=response, training_frame=prostate)

In [ ]:
gbm_grid.is_published()

In [ ]:
gbm_grid.publish()

In [ ]:
gbm_grid.is_published()

In [ ]:
gbm_grid.is_deployed()

In [ ]:
gbm_grid.deploy()

In [ ]:
gbm_grid.is_deployed()